code schema:

get list of airports in Europe - their IATA + ICAO code (key is ICAO)
web scraping - https://airmundo.com/en/airport-codes-european-airports/ 
    - this is for the purpose of calling airports 

for icao code, call the arrivals by airport and map the count of arrivals to the departure airport 

dictionary: 
    key: icao
        value: {key: icao of departure, value: count of arrivals}

making network graph:
for key in arrivals:
    add weighted direction from departure icao 
    each airport is a node
        add node for every airport that is mentioned
    each arrival is an edge 

centrality would be measured by number of arrivals (i.e. direction+weight)


# Only run the below code once to get the json data.

In [10]:
import requests
from bs4 import BeautifulSoup

#generates dictionary of european airports by their IACO and IATA codes 
page = requests.get("https://airmundo.com/en/airport-codes-european-airports/")
soup = BeautifulSoup(page.text, "html.parser")
body = soup.find_all("tr")

airport_codes = {}
i=1
while i<len(body):
    data = body[i].find_all("td")
    airport_codes[data[3].get_text(strip=True)] = data[2].get_text(strip=True)
    i+=1


In [11]:
import time

"""
Date range:
June 01 2024 - June 05 2024 or
1717200000 - 1717588799
"""

arrivals = []

"""takes a while to run. opensky has a 400 calls per day limit for anonymous users. there are about 264 airports in airport_codes. do not run more than once a day."""
for icao in airport_codes.keys():
    try:
        response = requests.get(f"https://opensky-network.org/api/flights/arrival?airport={icao}&begin=1717200000&end=1717588799")
        response.raise_for_status()  
    except requests.exceptions.HTTPError as error:
        """About 50 airports failed to receive a response for the period requested. See "not_included.txt" for more detail."""
        print(f"Error: {error}. {icao} will not be included.")
        continue 

    data = response.json()

    arrivals.append(data)
    time.sleep(1)



Error: 404 Client Error:  for url: https://opensky-network.org/api/flights/arrival?airport=LATI&begin=1717200000&end=1717588799. LATI will not be included.
Error: 404 Client Error:  for url: https://opensky-network.org/api/flights/arrival?airport=UDYZ&begin=1717200000&end=1717588799. UDYZ will not be included.


KeyboardInterrupt: 

In [ ]:
import json
import os

json_file = "data.json"

if os.path.exists(json_file):
    """write results to a json file"""
    with open(json_file, "w") as file:
        json.dump(arrivals, file, indent=4)
else:
    raise Exception("Please create a file titled 'data.json' in this directory to store the data in json format.")